In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import os, pickle
import sys
sys.path.append(".")
sys.path.append("../..")
import MinimaxParetoFair as MMPF

/Users/martinbertran/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


# Example of APStar algorithm on top of Linear Logistic Regression on Adult/Gender dataset

In [3]:
for split in range(5):
    
    #Load split data
    train_pd, val_pd, test_pd, col_tags = MMPF.dataset_loaders.load_adult(split)
    dataset_name = 'adult_gender'
    utility_tag = 'income-per-year_cat'
    secret_tag = 'sex_cat'

    x_train = train_pd[col_tags].values
    y_train = train_pd[utility_tag].values
    a_train = train_pd[secret_tag].values

    x_val = val_pd[col_tags].values
    y_val = val_pd[utility_tag].values
    a_val = val_pd[secret_tag].values

    x_test = test_pd[col_tags].values
    y_test = test_pd[utility_tag].values
    a_test = test_pd[secret_tag].values
    
    a_prior=np.zeros(a_train.max()+1)
    for a in range(a_train.max()+1):
        a_prior[a] = np.mean(a_train==a)
    
    #Instantiate logistic regression model
    model = MMPF.SKLearn_Weighted_LLR(x_train, y_train, a_train, x_val, y_val, a_val)

    # Run algorithm
    mua_ini = np.ones(a_val.max()+1)
    mua_ini /= mua_ini.sum()
    
    results = MMPF.APSTAR(model,mua_ini ,niter = 200 ,max_patience = 200 ,Kini=1,
                        Kmin = 20,alpha = 0.5,verbose = False)
    risk_list = results['risk_list']
    risk_best_list = results['risk_best_list']
    mu_list = results['mu_list']
    mu_best_list = results['mu_best_list'] 

    mu_best= mu_best_list[-1]
    model.weighted_fit(x_train,y_train,a_train, mu_best)
    
    # get classifier result tables
    #val tables
    val_pd['secret_gt']=val_pd[secret_tag]
    val_pd['utility_gt']=val_pd[utility_tag]
    pu = model.predict_proba(x_val)
    val_pd['utility_pest_0']=pu[:,0]
    val_pd['utility_pest_1']=pu[:,1]
    val_pd.drop(val_pd.columns.difference(
        ['secret_gt','utility_gt', 'utility_pest_0', 'utility_pest_1']), 1, inplace=True)

    #test tables
    test_pd['secret_gt']=test_pd[secret_tag]
    test_pd['utility_gt']=test_pd[utility_tag]
    pu = model.predict_proba(x_test)
    test_pd['utility_pest_0']=pu[:,0]
    test_pd['utility_pest_1']=pu[:,1]
    test_pd.drop(val_pd.columns.difference(
    ['secret_gt','utility_gt', 'utility_pest_0', 'utility_pest_1']), 1, inplace=True)

    

    #Save results
    save_dict={'df_val_result': val_pd, 'df_test_result':test_pd}
    save_dict['priors']={'sensitve': a_prior}
    save_dict['pareto_saves']=results
    
    
    save_dir = os.path.join('../results', dataset_name,dataset_name)
    os.makedirs(save_dir, exist_ok=True)
    with open( os.path.join(save_dir,'apstar_llr_{:d}.pkl'.format(split)),'wb') as f:
        save_dict={'df_val_result': val_pd, 'df_test_result':test_pd}
        pickle.dump(save_dict,f)

/Users/martinbertran/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1438: RuntimeWarning: divide by zero encountered in log
  return np.log(self.predict_proba(X))


patience , iterations 189 201
-----------------------------------------
patience , iterations 0 201
-----------------------------------------
patience , iterations 147 201
-----------------------------------------
patience , iterations 1 201
-----------------------------------------
patience , iterations 195 201
-----------------------------------------
